In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
a = pd.read_csv('C:/Users/Lenovo/Downloads/Loan/training_set (1).csv')

In [2]:
a.head(3)

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.0,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,NaN,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y


# check missing data

In [3]:
a.isna().sum()

Loan_ID               0
Gender               15
Married               3
Dependents           15
Education             1
Self_Employed        32
ApplicantIncome       2
CoapplicantIncome     1
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [4]:
from module import replacer,preprocessing,catconsep

In [5]:
replacer(a)

In [6]:
a.isna().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# Define x and y

In [7]:
y = a[["Loan_Status"]]
x = a.drop(labels=["Loan_ID","Loan_Status"],axis=1)

# Convert  'y'  in LabelEncoded format

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y1 = pd.DataFrame(le.fit_transform(y),columns=['Loan_Status'])

In [9]:
y1.head()

,Loan_Status
0,1
1,0
2,1
3,1
4,1


In [10]:
cat,con = catconsep(x)

In [11]:
cat

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area']

In [12]:
con

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

# Preprocessing(missing data treatment, standardization, OHE)

In [13]:
replacer(x)

In [14]:
x1 = preprocessing(x)

# Create a neural network

In [35]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [29]:
nn = Sequential()
nn.add(Dense(32,activation='relu',input_dim = x1.shape[1]))        # Input Layer
nn.add(Dense(32,activation='relu'))                              # Hidden Layer
nn.add(Dense(1,activation='sigmoid'))                          # Output Layer

In [31]:
nn.compile(loss = "binary_crossentropy",metrics=["accuracy"])
nn.fit(x1,y1,validation_split=0.2,epochs=30)

Epoch 1/30
16/16 [==============================] - 0s 21ms/step - loss: 0.6352 - accuracy: 0.6823 - val_loss: 0.6228 - val_accuracy: 0.6911
Epoch 2/30
16/16 [==============================] - 0s 2ms/step - loss: 0.5773 - accuracy: 0.7393 - val_loss: 0.5895 - val_accuracy: 0.7073
Epoch 3/30
16/16 [==============================] - 0s 2ms/step - loss: 0.5412 - accuracy: 0.7760 - val_loss: 0.5614 - val_accuracy: 0.7967
Epoch 4/30
16/16 [==============================] - 0s 2ms/step - loss: 0.5134 - accuracy: 0.7943 - val_loss: 0.5345 - val_accuracy: 0.8049
Epoch 5/30
16/16 [==============================] - 0s 2ms/step - loss: 0.4890 - accuracy: 0.8045 - val_loss: 0.5149 - val_accuracy: 0.8130
Epoch 6/30
16/16 [==============================] - 0s 2ms/step - loss: 0.4753 - accuracy: 0.8065 - val_loss: 0.5041 - val_accuracy: 0.7886
Epoch 7/30
16/16 [==============================] - 0s 3ms/step - loss: 0.4629 - accuracy: 0.8045 - val_loss: 0.4993 - val_accuracy: 0.7967
Epoch 8/30
16/16 [=

# keras tuner

In [34]:
def model_builder(hp):
    nn = Sequential()
    nn.add(Dense(32, activation = 'relu', input_dim = x1.shape[1]))
    x = hp.Int('units', min_value = 32, max_value = 512, step= 32)
    nn.add(Dense(units=x))
    lr = hp.Choice('rate', values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
    nn.add(Dropout(rate = lr))
    nn.add(Dense(1,activation='sigmoid'))
    nn.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer = 'Adam')
    return nn

In [37]:
import keras_tuner as kt
tuner = kt.Hyperband(model_builder, objective='val_accuracy', max_epochs=10,
                    directory= 'C:/Users/Lenovo/Downloads/kt5',
                     project_name = 'loan_status')

In [38]:
es = EarlyStopping(patience=20, monitor = 'val_loss')

In [39]:
tuner.search(x1, y1, callbacks = [es], validation_split= 0.2, epochs = 50)

Trial 30 Complete [00h 00m 01s]
val_accuracy: 0.8292682766914368

Best val_accuracy So Far: 0.8373983502388
Total elapsed time: 00h 00m 40s
INFO:tensorflow:Oracle triggered exit


In [45]:
tuner.get_best_hyperparameters(num_trials=1)[0].get('units')

352

In [48]:
tuner.get_best_hyperparameters()[0].get('rate')

0.5

In [50]:
# Gradient Descent

nn = Sequential()
nn.add(Dense(32,activation='relu',input_dim = x1.shape[1]))       
nn.add(Dense(352,activation='relu')) 
nn.add(Dropout(0.5))
nn.add(Dense(1,activation='sigmoid'))  

nn.compile(loss='binary_crossentropy', metrics=['accuracy'])
nn.fit(x1,y1, validation_split=0.2, epochs=100, batch_size=100)

Epoch 1/100
5/5 [==============================] - 0s 51ms/step - loss: 0.6358 - accuracy: 0.6904 - val_loss: 0.5959 - val_accuracy: 0.6992
Epoch 2/100
5/5 [==============================] - 0s 9ms/step - loss: 0.5727 - accuracy: 0.7352 - val_loss: 0.5536 - val_accuracy: 0.7561
Epoch 3/100
5/5 [==============================] - 0s 8ms/step - loss: 0.5271 - accuracy: 0.7821 - val_loss: 0.5251 - val_accuracy: 0.8211
Epoch 4/100
5/5 [==============================] - 0s 8ms/step - loss: 0.5022 - accuracy: 0.7984 - val_loss: 0.5064 - val_accuracy: 0.8211
Epoch 5/100
5/5 [==============================] - 0s 8ms/step - loss: 0.4904 - accuracy: 0.8004 - val_loss: 0.4937 - val_accuracy: 0.8293
Epoch 6/100
5/5 [==============================] - 0s 8ms/step - loss: 0.4825 - accuracy: 0.8045 - val_loss: 0.4858 - val_accuracy: 0.8374
Epoch 7/100
5/5 [==============================] - 0s 9ms/step - loss: 0.4711 - accuracy: 0.7984 - val_loss: 0.4810 - val_accuracy: 0.8374
Epoch 8/100
5/5 [=========

In [51]:
# Adam

nn = Sequential()
nn.add(Dense(32,activation='relu',input_dim = x1.shape[1]))       
nn.add(Dense(352,activation='relu')) 
nn.add(Dropout(0.5))
nn.add(Dense(1,activation='sigmoid'))  

nn.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
nn.fit(x1,y1, validation_split=0.2, epochs=100)

Epoch 1/100
16/16 [==============================] - 0s 19ms/step - loss: 0.6147 - accuracy: 0.6864 - val_loss: 0.5866 - val_accuracy: 0.6829
Epoch 2/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5398 - accuracy: 0.7413 - val_loss: 0.5378 - val_accuracy: 0.8130
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4855 - accuracy: 0.7984 - val_loss: 0.4984 - val_accuracy: 0.8130
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4694 - accuracy: 0.8045 - val_loss: 0.4905 - val_accuracy: 0.8130
Epoch 5/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4627 - accuracy: 0.8045 - val_loss: 0.4900 - val_accuracy: 0.8130
Epoch 6/100
16/16 [==============================] - 0s 6ms/step - loss: 0.4423 - accuracy: 0.8024 - val_loss: 0.5003 - val_accuracy: 0.8049
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.4398 - accuracy: 0.8024 - val_loss: 0.5135 - val_accuracy: 0.8049
Epoch 8/100


# Testing set

In [52]:
b = pd.read_csv('C:/Users/Lenovo/Downloads/Loan/testing_set (1).csv')

In [53]:
b = b.drop(labels=["Loan_ID"],axis=1)

In [54]:
replacer(b)

In [55]:
b1 = preprocessing(b)

In [56]:
b1.shape

(367, 20)

In [57]:
nn.predict_classes(b1)

array([[1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
    

In [58]:
le.inverse_transform(nn.predict_classes(b1))

array(['Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'N',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'Y', 'N', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'N',
       'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'N', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'N', 'N', 'N', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'N',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'N',
       'N', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

In [59]:
b["Predicted_loan_status"] = le.inverse_transform(nn.predict_classes(b1))

In [60]:
b

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Predicted_loan_status
0,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.000000,Urban,Y
1,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.000000,Urban,Y
2,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.000000,Urban,Y
3,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,0.825444,Urban,Y
4,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.000000,Urban,N
...,...,...,...,...,...,...,...,...,...,...,...,...
362,Male,Yes,3+,Not Graduate,Yes,4009,1777,113.0,360.0,1.000000,Urban,Y
363,Male,Yes,0,Graduate,No,4158,709,115.0,360.0,1.000000,Urban,Y
364,Male,No,0,Graduate,No,3250,1993,126.0,360.0,0.825444,Semiurban,Y
365,Male,Yes,0,Graduate,No,5000,2393,158.0,360.0,1.000000,Rural,Y


# Problem Statement 2: Loan Amount Prediction

In [62]:
# data preparation

a1 = a[a['Loan_Status']=='Y']
a1 = a1.drop(labels=['Loan_ID'],axis=1)

In [63]:
a2 = b[b['Predicted_loan_status']=='Y']

In [64]:
a1.shape

(422, 12)

In [65]:
a2.shape

(266, 12)

In [66]:
a1.columns = a2.columns

In [67]:
c = pd.concat([a1,a2])

In [68]:
c.isna().sum()

Gender                   0
Married                  0
Dependents               0
Education                0
Self_Employed            0
ApplicantIncome          0
CoapplicantIncome        0
LoanAmount               0
Loan_Amount_Term         0
Credit_History           0
Property_Area            0
Predicted_loan_status    0
dtype: int64

In [72]:
print(c.shape)
c.index = range(0,688,1)
c.head()


(688, 12)


,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Predicted_loan_status
0,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.0,Urban,Y
1,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.0,Urban,Y
2,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.0,Urban,Y
3,Male,No,0,Graduate,No,6000.0,0.0,141.000000,360.0,1.0,Urban,Y
4,Male,Yes,2,Graduate,Yes,5417.0,4196.0,267.000000,360.0,1.0,Urban,Y


# Preprocessing

In [73]:
y = c[['LoanAmount']]
x = c.drop(labels=['LoanAmount'],axis=1)

In [74]:
replacer(y)
replacer(x)

In [75]:
x1 = preprocessing(x)

In [76]:
x1.isna().sum()

ApplicantIncome            0
CoapplicantIncome          0
Loan_Amount_Term           0
Credit_History             0
Gender_Female              0
Gender_Male                0
Married_No                 0
Married_Yes                0
Dependents_0               0
Dependents_1               0
Dependents_2               0
Dependents_3+              0
Education_Graduate         0
Education_Not Graduate     0
Self_Employed_No           0
Self_Employed_Yes          0
Property_Area_Rural        0
Property_Area_Semiurban    0
Property_Area_Urban        0
Predicted_loan_status_Y    0
dtype: int64

# Create NN

In [78]:
from tensorflow.keras.losses import mae, mean_squared_error, mean_absolute_error

In [82]:
nn1 = Sequential()
nn1.add(Dense(32,activation = 'relu', input_dim = x1.shape[1]))
nn1.add(Dense(32,activation = 'relu'))
nn1.add(Dense(1))

nn1.compile(loss = 'mean_squared_error')
nn1.fit(x1, y, validation_split=0.2, epochs=50)

Epoch 1/50
18/18 [==============================] - 0s 13ms/step - loss: 26945.8945 - val_loss: 18739.0664
Epoch 2/50
18/18 [==============================] - 0s 3ms/step - loss: 26558.3711 - val_loss: 18359.8184
Epoch 3/50
18/18 [==============================] - 0s 3ms/step - loss: 26044.3477 - val_loss: 17819.0977
Epoch 4/50
18/18 [==============================] - 0s 3ms/step - loss: 25322.7461 - val_loss: 17051.6055
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 24333.8789 - val_loss: 16044.9062
Epoch 6/50
18/18 [==============================] - 0s 3ms/step - loss: 23041.2637 - val_loss: 14769.8574
Epoch 7/50
18/18 [==============================] - 0s 3ms/step - loss: 21448.1621 - val_loss: 13202.9180
Epoch 8/50
18/18 [==============================] - 0s 3ms/step - loss: 19551.5527 - val_loss: 11491.5977
Epoch 9/50
18/18 [==============================] - 0s 3ms/step - loss: 17455.0801 - val_loss: 9577.7832
Epoch 10/50
18/18 [===========================

In [225]:
# keras tuner

def model_builder(hp):
    nn = Sequential()
    nn.add(Dense(32, activation = 'relu', input_dim = x1.shape[1]))
    x = hp.Int('units', min_value = 32, max_value = 512, step= 32)
    nn.add(Dense(units=x))
    lr = hp.Choice('rate', values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
    nn.add(Dropout(rate = lr))
    nn.add(Dense(1,activation='relu'))
    nn.compile(loss='mean_squared_error')
    return nn

In [226]:
tuner = kt.Hyperband(model_builder, objective='val_loss', max_epochs=10,
                    directory= 'C:/Users/Lenovo/Downloads/kt6',
                     project_name = 'loan_status_2')

INFO:tensorflow:Reloading Oracle from existing project C:/Users/Lenovo/Downloads/kt6\loan_status_2\oracle.json


In [227]:
tuner.search(x1, y, validation_split = 0.2, epochs= 50)

Trial 30 Complete [00h 00m 01s]
val_loss: 2209.460205078125

Best val_loss So Far: 1964.0677490234375
Total elapsed time: 00h 00m 44s
INFO:tensorflow:Oracle triggered exit


In [228]:
tuner.get_best_hyperparameters(num_trials= 1)[0].get('units')

448

In [229]:
tuner.get_best_hyperparameters()[0].get('rate')

0.5

In [230]:
nn1 = Sequential()
nn1.add(Dense(32,activation = 'relu', input_dim = x1.shape[1]))
nn1.add(Dense(448,activation = 'relu'))
nn1.add(Dropout(0.5))
nn1.add(Dense(1))

nn1.compile(loss = 'mean_squared_error')
nn1.fit(x1, y, validation_split=0.2, epochs=50)

Epoch 1/50
18/18 [==============================] - 0s 15ms/step - loss: 26175.2695 - val_loss: 17088.3730
Epoch 2/50
18/18 [==============================] - 0s 3ms/step - loss: 23340.2461 - val_loss: 13979.4424
Epoch 3/50
18/18 [==============================] - 0s 3ms/step - loss: 19136.2754 - val_loss: 9795.3721
Epoch 4/50
18/18 [==============================] - 0s 3ms/step - loss: 13732.1992 - val_loss: 5477.2383
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 9060.6719 - val_loss: 2980.2878
Epoch 6/50
18/18 [==============================] - 0s 4ms/step - loss: 6515.1484 - val_loss: 2429.3247
Epoch 7/50
18/18 [==============================] - 0s 4ms/step - loss: 5768.0630 - val_loss: 2400.7266
Epoch 8/50
18/18 [==============================] - 0s 4ms/step - loss: 5351.7974 - val_loss: 2285.8438
Epoch 9/50
18/18 [==============================] - 0s 4ms/step - loss: 5253.0420 - val_loss: 2232.6172
Epoch 10/50
18/18 [==============================] - 0s 4

# prepare data for prediction 

In [231]:
b1 = b[b['Predicted_loan_status'] == 'N']

In [232]:
replacer(b1)

In [233]:
b1.shape

(101, 12)

In [234]:
b1.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area',
       'Predicted_loan_status'],
      dtype='object')

In [235]:
x.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'Loan_Amount_Term',
       'Credit_History', 'Property_Area', 'Predicted_loan_status'],
      dtype='object')

In [236]:
x.shape

(688, 11)

In [237]:
b1 = b1.drop(labels=['LoanAmount'],axis=1)

In [238]:
b1.columns = x.columns

In [239]:
newX = preprocessing(b1)

In [240]:
replacer(newX)

In [241]:
newX.isna().sum()

ApplicantIncome            0
CoapplicantIncome          0
Loan_Amount_Term           0
Credit_History             0
Gender_Female              0
Gender_Male                0
Married_No                 0
Married_Yes                0
Dependents_0               0
Dependents_1               0
Dependents_2               0
Dependents_3+              0
Education_Graduate         0
Education_Not Graduate     0
Self_Employed_No           0
Self_Employed_Yes          0
Property_Area_Rural        0
Property_Area_Semiurban    0
Property_Area_Urban        0
Predicted_loan_status_N    0
dtype: int64

In [242]:
newX.columns = x1.columns

In [243]:
# Predictions

In [244]:
pred = nn1.predict(newX)

In [245]:
pred

array([[102.44804 ],
       [114.15524 ],
       [ 99.32795 ],
       [243.42567 ],
       [138.84703 ],
       [226.30446 ],
       [100.668365],
       [135.0791  ],
       [137.55927 ],
       [ 93.28738 ],
       [ 94.78196 ],
       [ 90.19334 ],
       [122.92066 ],
       [129.25049 ],
       [106.111885],
       [119.33306 ],
       [183.24959 ],
       [151.04948 ],
       [ 99.573326],
       [133.269   ],
       [137.37807 ],
       [119.34319 ],
       [ 83.0676  ],
       [ 84.77216 ],
       [100.64621 ],
       [ 81.103355],
       [128.96945 ],
       [261.57098 ],
       [162.39421 ],
       [100.1352  ],
       [129.748   ],
       [169.8879  ],
       [200.04823 ],
       [101.181526],
       [159.36801 ],
       [178.01593 ],
       [ 97.236206],
       [ 89.34757 ],
       [ 84.65835 ],
       [ 76.774216],
       [ 93.03472 ],
       [107.30529 ],
       [136.31819 ],
       [ 88.62047 ],
       [ 97.581795],
       [138.17847 ],
       [122.43281 ],
       [264.7

# Problem Statement 3 : loan term

In [198]:
train = a
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849.00000,0.0,146.412162,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,5405.54085,1508.0,128.000000,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000.00000,0.0,66.000000,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583.00000,2358.0,120.000000,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000.00000,0.0,141.000000,360.0,1.0,Urban,Y


In [199]:
test = b
test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Predicted_loan_status
0,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.000000,Urban,Y
1,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.000000,Urban,Y
2,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.000000,Urban,Y
3,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,0.825444,Urban,Y
4,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.000000,Urban,N


In [200]:
p1 = train[(train['Loan_Status'] == 'Y') & (train['Loan_Amount_Term'] <= 240)]
p2 = test[(test['Predicted_loan_status'] == 'Y') & (test['Loan_Amount_Term'] <= 240)]

In [201]:
p1 = p1.drop(labels=['Loan_ID'],axis=1)

In [202]:
p1.shape

(41, 12)

In [203]:
p2.shape

(27, 12)

In [204]:
replacer(p1)
replacer(p2)

In [205]:
p1.columns = p2.columns

In [206]:
p = pd.concat([p1,p2])

In [207]:
p.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Predicted_loan_status
14,Male,Yes,2,Graduate,No,1299.0,1086.0,17.0,120.0,1.000000,Urban,Y
16,Male,No,1,Not Graduate,No,3596.0,0.0,100.0,240.0,0.842199,Urban,Y
68,Male,Yes,3+,Not Graduate,Yes,7100.0,0.0,125.0,60.0,1.000000,Urban,Y
84,Male,Yes,1,Graduate,No,3988.0,0.0,50.0,240.0,1.000000,Urban,Y
91,Male,Yes,2,Graduate,No,6250.0,5654.0,188.0,180.0,1.000000,Semiurban,Y


In [208]:
p.shape

(68, 12)

In [209]:
cat3 , con3 = catconsep(p)

In [210]:
cat3

['Gender',
 'Married',
 'Dependents',
 'Education',
 'Self_Employed',
 'Property_Area',
 'Predicted_loan_status']

In [211]:
con3

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

In [212]:
#define x and y

x3 = p.drop(labels=["Loan_Amount_Term"],axis=1)
y3 = p[["Loan_Amount_Term"]]

In [213]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in cat3:
    x3[i] = le.fit_transform(x3[i])

In [214]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x3, y3, test_size=0.2, random_state=31)

# create NN

In [215]:
nn1 = Sequential()
nn1.add(Dense(32,activation = 'relu', input_dim = x3.shape[1]))
nn1.add(Dense(32,activation = 'relu'))
nn1.add(Dense(1))

nn1.compile(loss = 'mean_squared_error')
nn1.fit(x3, y3, validation_split=0.2, epochs=50)

Epoch 1/50
2/2 [==============================] - 0s 94ms/step - loss: 305039.1562 - val_loss: 44463.3867
Epoch 2/50
2/2 [==============================] - 0s 14ms/step - loss: 48764.3086 - val_loss: 25522.8262
Epoch 3/50
2/2 [==============================] - 0s 15ms/step - loss: 30157.7617 - val_loss: 20149.0957
Epoch 4/50
2/2 [==============================] - 0s 13ms/step - loss: 24003.0176 - val_loss: 14730.7080
Epoch 5/50
2/2 [==============================] - 0s 15ms/step - loss: 21258.6426 - val_loss: 12629.4199
Epoch 6/50
2/2 [==============================] - 0s 18ms/step - loss: 19599.2676 - val_loss: 13464.5674
Epoch 7/50
2/2 [==============================] - 0s 14ms/step - loss: 17886.1855 - val_loss: 10499.8076
Epoch 8/50
2/2 [==============================] - 0s 13ms/step - loss: 19545.9121 - val_loss: 14574.3232
Epoch 9/50
2/2 [==============================] - 0s 13ms/step - loss: 19446.4668 - val_loss: 9570.9541
Epoch 10/50
2/2 [==============================] - 0s 1

In [246]:
# keras tuner

def model_builder(hp):
    nn = Sequential()
    nn.add(Dense(32, activation = 'relu', input_dim = x3.shape[1]))
    x = hp.Int('units', min_value = 32, max_value = 512, step= 32)
    nn.add(Dense(units=x))
    lr = hp.Choice('rate', values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
    nn.add(Dropout(rate = lr))
    nn.add(Dense(1))
    nn.compile(loss='mean_squared_error')
    return nn

In [249]:
tuner = kt.Hyperband(model_builder, objective='val_loss', max_epochs=10,
                    directory= 'C:/Users/Lenovo/Downloads/kt6',
                     project_name = 'loan_status_3')

In [250]:
tuner.search(x3, y3, validation_split = 0.2, epochs= 50)

Trial 30 Complete [00h 00m 01s]
val_loss: 8851.1669921875

Best val_loss So Far: 7615.88330078125
Total elapsed time: 00h 00m 39s
INFO:tensorflow:Oracle triggered exit


In [251]:
tuner.get_best_hyperparameters(num_trials=1)[0].get('units')

256

In [252]:
tuner.get_best_hyperparameters()[0].get('rate')

0.3

In [253]:
nn1 = Sequential()
nn1.add(Dense(32,activation = 'relu', input_dim = x3.shape[1]))
nn1.add(Dense(256,activation = 'relu'))
nn1.add(Dropout(0.3))
nn1.add(Dense(1))

nn1.compile(loss = 'mean_squared_error')
nn1.fit(x3, y3, validation_split=0.2, epochs=50)

Epoch 1/50
2/2 [==============================] - 0s 118ms/step - loss: 3007912.0000 - val_loss: 19610.2852
Epoch 2/50
2/2 [==============================] - 0s 15ms/step - loss: 434443.1562 - val_loss: 86410.0078
Epoch 3/50
2/2 [==============================] - 0s 14ms/step - loss: 258002.1406 - val_loss: 19307.8340
Epoch 4/50
2/2 [==============================] - 0s 13ms/step - loss: 320523.0625 - val_loss: 36710.0000
Epoch 5/50
2/2 [==============================] - 0s 14ms/step - loss: 193506.8906 - val_loss: 13391.8740
Epoch 6/50
2/2 [==============================] - 0s 13ms/step - loss: 329772.8125 - val_loss: 46081.4922
Epoch 7/50
2/2 [==============================] - 0s 15ms/step - loss: 416805.6562 - val_loss: 11095.9932
Epoch 8/50
2/2 [==============================] - 0s 13ms/step - loss: 99935.9922 - val_loss: 8695.1660
Epoch 9/50
2/2 [==============================] - 0s 12ms/step - loss: 264556.0312 - val_loss: 13872.4189
Epoch 10/50
2/2 [=============================

In [254]:
# testing data

In [255]:
q = test[(test['Predicted_loan_status'] == 'N') & (test['Loan_Amount_Term'] <= 240)]

In [256]:
q1 = q.drop(labels=['Loan_Amount_Term'],axis=1)

In [257]:
q1

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History,Property_Area,Predicted_loan_status
84,Male,Yes,0,Graduate,No,4456,0,131.0,0.000000,Semiurban,N
164,Male,No,1,Graduate,No,3180,2370,80.0,0.825444,Rural,N
173,Male,No,0,Graduate,No,3017,2845,159.0,0.000000,Urban,N
190,Female,Yes,1,Graduate,No,4333,0,132.0,1.000000,Rural,N
338,Male,Yes,1,Graduate,Yes,2360,3355,87.0,1.000000,Rural,N
342,Male,Yes,3+,Not Graduate,No,3522,0,81.0,1.000000,Rural,N
354,Male,Yes,3+,Not Graduate,No,5316,187,158.0,0.000000,Semiurban,N


In [258]:
cat4, con4 = catconsep(q1)

In [259]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for i in cat4:
    q1[i] = le.fit_transform(q1[i])

In [260]:
q1

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Credit_History,Property_Area,Predicted_loan_status
84,1,1,0,0,0,4456,0,131.0,0.000000,1,0
164,1,0,1,0,0,3180,2370,80.0,0.825444,0,0
173,1,0,0,0,0,3017,2845,159.0,0.000000,2,0
190,0,1,1,0,0,4333,0,132.0,1.000000,0,0
338,1,1,1,0,1,2360,3355,87.0,1.000000,0,0
342,1,1,2,1,0,3522,0,81.0,1.000000,0,0
354,1,1,2,1,0,5316,187,158.0,0.000000,1,0


In [261]:
q1.shape

(7, 11)

In [262]:
replacer(q1)

# Predictions

In [263]:
pred = nn1.predict(q1)

In [264]:
pred

array([[149.28183],
       [205.0286 ],
       [212.25166],
       [145.0659 ],
       [167.32556],
       [118.03263],
       [178.58362]], dtype=float32)